In [1]:
!pip install great_expectations==1.3.0 pandas opentelemetry-api opentelemetry-distro opentelemetry-exporter-otlp #opentelemetry-exporter-otlp-proto-grpc  
!pip install opentelemetry-semantic-conventions opentelemetry-util-http wrapt opentelemetry-exporter-otlp-proto-http

import os, subprocess

In [2]:
DATA_PRODUCT_NAME = 'consuntiviDiProduzione'
QUALITY_SIDECAR_SCRIPT_BASE_PATH = 'quality_sidecar/'
EXPECTATIONS_JSON_FILE_PATH = 'resources/gx_v0.1.json'

In [4]:
# OTLP variables. DO NOT REMOVE!
os.environ['DATA_PRODUCT_NAME'] = DATA_PRODUCT_NAME
os.environ['OTEL_EXPORTER_OTLP_ENDPOINT'] = 'http://platform-collector:4318'
os.environ['OTEL_EXPORTER_OTLP_PROTOCOL'] = "http/protobuf"
os.environ['OTEL_SERVICE_NAME'] = f'{os.getenv("DATA_PRODUCT_NAME")}-quality_sidecar'

command = f"opentelemetry-instrument --metrics_exporter otlp,console --logs_exporter otlp,console python {QUALITY_SIDECAR_SCRIPT_BASE_PATH}/app.py {QUALITY_SIDECAR_SCRIPT_BASE_PATH}/{EXPECTATIONS_JSON_FILE_PATH}"
subprocess.run(command, shell=True, check=True)

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmp8aas2qtr' for ephemeral docs site
INFO:root:Starting the application...
INFO:root:Reading GreatExpectations json file...
INFO:root:Setting up GreatExpectations...


INFO:root:Creating ValidationResults...
Calculating Metrics:  14%|█▎        | 3/22 [00:00<00:00, 84.31it/s] /opt/conda/lib/python3.11/site-packages/great_expectations/expectations/metrics/column_map_metrics/column_values_match_regex.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return column.astype(str).str.contains(regex)
Calculating Metrics: 100%|██████████| 22/22 [00:00<00:00, 206.88it/s]
INFO:root:ValidationResults created!
INFO:root:Creating ValidationResults...
Calculating Metrics:   9%|▉         | 2/22 [00:00<00:00, 58.59it/s] 

{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "dataSourceSample-dataAssetSample",
          "column": "vendor_id"
        },
        "meta": {
          "expectation_name": "checkNotNull",
          "data_product_name": "consuntiviDiProduzione"
        },
        "id": "d3243727-a938-4b5a-b24a-ac8a6a993440"
      },
      "result": {
        "element_count": 10000,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "partial_unexpected_counts": [],
        "partial_unexpected_index_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": false,
      "expectation_config": {
        "type": "expect_column_values_to_be_between",
       

Calculating Metrics:  14%|█▎        | 3/22 [00:00<00:00, 53.60it/s]/opt/conda/lib/python3.11/site-packages/great_expectations/expectations/metrics/column_map_metrics/column_values_match_regex.py:25: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return column.astype(str).str.contains(regex)
Calculating Metrics: 100%|██████████| 22/22 [00:00<00:00, 172.27it/s]
INFO:root:ValidationResults created!


CompletedProcess(args='opentelemetry-instrument --metrics_exporter otlp,console --logs_exporter otlp,console python quality_sidecar//app.py quality_sidecar//resources/gx_v0.1.json', returncode=0)